# Exploratory Data Analysis (EDA)

**Author:** Brendon McNulty  
**Date:** September 30, 2025  

**Purpose:**  
This notebook performs a custom exploratory data analysis project using Python, pandas, matplotlib, and seaborn. The goal is to explore a dataset, analyze its structure, and tell a data story through visualizations.


In [1]:
# Import required libraries for data analysis and visualization
import pandas as pd        # Data manipulation and analysis
import seaborn as sns      # Statistical data visualization
import matplotlib.pyplot as plt  # Core plotting library

